In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [16]:
df = pd.read_csv(r"C:\Users\User\Desktop\Machine Learning\ML_Task_01\dataset_combined.csv")
print(df)

                                                  Path     Identity  \
0    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
1    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
2    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
3    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
4    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
..                                                 ...          ...   
194  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
195  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
196  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
197  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
198  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   

      Expression  Age  
0        Smiling   23  
1    Not Smiling   23  
2    Not Smiling   23  
3        Smiling   23  
4    Not Smiling   23  
.. 

In [17]:
train, test = train_test_split(df, test_size=0.3, random_state = 8)

In [18]:
train.loc[train['Age'] == 23].describe()

,Age
count,73.0
mean,23.0
std,0.0
min,23.0
25%,23.0
50%,23.0
75%,23.0
max,23.0


In [19]:
X_data = train['Path']
X_tdata = test['Path']
y1_train = train['Identity']
y2_train = train['Expression']
y3_train = train['Age']
y1_test = test['Identity']
y2_test = test['Expression']
y3_test = test['Age']

In [20]:
import skimage
from skimage import io
from skimage import transform

lst = []
for path in X_data:
    curr_image = io.imread(path)
    img_resize = transform.resize(curr_image, output_shape=(32,32,3)) #resize image to match a certain type
    img_reshape = np.reshape(img_resize, (32*32*3))
    lst.append(img_reshape/255.)
    
X_train = np.array(lst)
print(X_train)

[[0.00020528 0.00021926 0.00024333 ... 0.00062626 0.00068319 0.00088945]
 [0.00118929 0.00121232 0.00106665 ... 0.00080002 0.00084119 0.00091485]
 [0.00223232 0.00152865 0.00110605 ... 0.00096045 0.00081889 0.00078292]
 ...
 [0.00133537 0.00099903 0.00115933 ... 0.00213558 0.00232012 0.00253542]
 [0.0006189  0.00088462 0.00140921 ... 0.00023611 0.00023125 0.00039626]
 [0.00024174 0.00024354 0.0002437  ... 0.00065935 0.00061324 0.00053635]]


In [21]:
lst = []
for path in X_tdata:
    curr_image = io.imread(path)
    img_resize = transform.resize(curr_image, output_shape=(32,32,3))
    img_reshape = np.reshape(img_resize, (32*32*3))
    lst.append(img_reshape/255.)
    
X_test = np.array(lst)
print(X_test)

[[0.00372479 0.0036633  0.00364792 ... 0.00092841 0.00082061 0.00067896]
 [0.00369036 0.00335115 0.00285218 ... 0.0009648  0.00092806 0.00090443]
 [0.00148349 0.00128357 0.00114516 ... 0.00103969 0.0008479  0.00067647]
 ...
 [0.0006255  0.00047171 0.00045633 ... 0.00056502 0.00045768 0.00056406]
 [0.00270232 0.00256153 0.00237478 ... 0.00047237 0.00053545 0.00058865]
 [0.00389981 0.00390084 0.00388917 ... 0.00392059 0.00392001 0.0039204 ]]


In [22]:
input_layer = keras.layers.Input(shape=(32*32*3,))
X = keras.layers.Dense(1, activation='relu')(input_layer)
model = keras.Model(inputs = input_layer, outputs = X)

In [23]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3072)]            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 3073      
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint1.h5',  # Filepath to save the model
    monitor='loss',  # Metric to monitor 
    save_best_only=True,  # Save only the best model
    save_weights_only=False,  # Save the entire model (including architecture)
    mode='min'  # Mode for comparison (minimizing the monitored metric)
)

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.mean_squared_error)
model.fit(X_train, y3_train, epochs=50, batch_size=8, callbacks=[checkpoint_callback])

preds = model.predict(X_test)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, y3_test)

print("MSE:", loss)

Epoch 1/50
18/18 [==============================] - 1s 5ms/step - loss: 2029.4047
Epoch 2/50
18/18 [==============================] - 0s 4ms/step - loss: 2020.9110
Epoch 3/50
18/18 [==============================] - 0s 5ms/step - loss: 2012.6028
Epoch 4/50
18/18 [==============================] - 0s 6ms/step - loss: 2004.1025
Epoch 5/50
18/18 [==============================] - 0s 5ms/step - loss: 1995.8258
Epoch 6/50
18/18 [==============================] - 0s 5ms/step - loss: 1987.5065
Epoch 7/50
18/18 [==============================] - 0s 4ms/step - loss: 1979.3608
Epoch 8/50
18/18 [==============================] - 0s 5ms/step - loss: 1971.2601
Epoch 9/50
18/18 [==============================] - 0s 4ms/step - loss: 1963.0032
Epoch 10/50
18/18 [==============================] - 0s 5ms/step - loss: 1954.8640
Epoch 11/50
18/18 [==============================] - 0s 4ms/step - loss: 1946.7244
Epoch 12/50
18/18 [==============================] - 0s 4ms/step - loss: 1938.6187
Epoch 13/50
1

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)